In [ ]:
import numpy as np
import pandas as pd
from scipy.spatial import distance
from nolds import lyap_r
import empyrical as ep
import matplotlib.pyplot as plt

In [ ]:
univ01 = ['AG', 'AL', 'NI', 'ZN',
        'RB', 'HC', 'I', 'J', 'JM', 'FG',
        'BU', 'L', 'MA', 'PP', 'RU', 'V', 'TA', 'FU', 'EG', 'SP',
        'A', 'M', 'P', 'OI', 'Y', 'SR', 'CF', 'C', 'RM']

univ02 = ['AG', 'AL', 'NI', 'ZN',
            'RB', 'HC', 'I', 'J', 'JM', 'FG',
            'BU', 'L', 'MA', 'PP', 'RU', 'V', 'TA', 'FU', 'EG', 'SP',
            'A', 'M', 'P', 'OI', 'Y', 'SR', 'CF', 'C', 'RM','CU','PB','SN','SF','SM','AP','SA']

univ_nonferrous = ['CU','AL','NI','ZN','PB','SN']
univ_black = ['RB','I','HC','J','JM','FG','SF','SM','SA']
univ_chemic = ['FU','BU','RU','L','TA','V','EG','MA','PP']
univ_agri = ['A','AP','C', 'CF', 'CS', 'JD', 'M', 'OI', 'P', 'RM', 'SR', 'Y']
univ_precious = ['AU','AG']

In [ ]:
#行情数据
sec_post_1d_df = pd.read_pickle(r'/Users/tiancaixiaohuoban/Desktop/实习/买方实习/中信建投期货（金融工程）/策略设计/dom_cont_post_1d')
sec_post_1d_df

In [ ]:
# 生成交易信号
def generate_signal_for_df(df, ma_windows, slope_window):
    # 计算斜率
    df['Slope'] = df['lyp'].diff(slope_window)
    # 使用均线辅助判断趋势方向，这里使用简单移动平均
    # 计算均线
    for ma_window in ma_windows:
        df[f'MA_{ma_window}'] = df['close'].rolling(window=ma_window).mean()
    # 清除含空值的行
    # 构造基础交易信号
    # 当长期均线低于短期均线时，标记为潜在进场时机，即1.0，否则为0
    df['signal'] = np.where(df['MA_12'] > df['MA_30'], 1.0, 0.0)
    result = df
    for i in range(1,len(result)):
        row_index = result.index[i]
        # 系统混沌程度出现下降趋势，市场行情明朗，且具有潜在进场机会，说明应该买入
        if (result.loc[row_index,'Slope'] < 0) and (result.loc[row_index,'signal'] == 1):
             result.loc[row_index,'Signal'] = 1
        # 系统混沌程度出现下降趋势，市场行情明朗，且具有下行趋势，说明应该卖出
        if (result.loc[row_index,'Slope'] < 0) and (result.loc[row_index,'signal'] == 0):
             result.loc[row_index,'Signal'] = -1
    result['Signal'] = result['Signal'].fillna(0)
    return result['Signal']

In [ ]:
# 指定计算李雅普诺夫指数的时间延迟和嵌入维度
time_delay = 10
embedding_dimension = 1
# 制定均线计算窗口
ma_windows = [12, 18, 24, 30]
# 设定斜率计算窗口
slope_window = 144

In [ ]:
# 循环产生信号
for asset in univ01:
    print(asset)
    df = sec_post_1d_df.loc[asset].T
    if (asset=='AG'):
        # 计算最大李雅普诺夫指数
        df['lyp'] = df['close'].rolling(window=144).apply(lambda x: lyap_r(x,emb_dim=embedding_dimension, lag=time_delay))
        test_signal_df = generate_signal_for_df(df, ma_windows, slope_window)
         # 产生信号
        continue
    # 计算最大李雅普诺夫指数
    df['lyp'] = df['close'].rolling(window=144).apply(lambda x: lyap_r(x,emb_dim=embedding_dimension, lag=time_delay))
    df_signal = generate_signal_for_df(df, ma_windows, slope_window)
    test_signal_df = pd.concat([test_signal_df, df_signal], axis=1)

In [ ]:
test_signal_df = test_signal_df.fillna(0)
test_signal_df.columns = univ01
test_signal_df = test_signal_df.T

In [ ]:
test_signal_df

In [ ]:
test_sig_df = test_signal_df

In [ ]:
#收益率序列
pct_df = sec_post_1d_df.xs('close',level=1).pct_change(axis=1).reindex(test_sig_df.index)
#手续费
fee_df = abs(test_sig_df.diff(axis=1).fillna(0))*1e-4 #单边万一的手续费
#当天的信号作用到第二天，用close计算出来的收益率，shift-1.  
pnl_seires = ((pct_df.shift(-1,axis=1).fillna(0)*test_sig_df-fee_df).mean()+1).cumprod()
pnl_seires.plot()

In [ ]:
return_df = (pct_df.shift(-1,axis=1).fillna(0)*test_sig_df-fee_df).mean()
#通过Empyrical 计算夏普等绩效指标。无风险利率取0% 
total_return = ep.cum_returns_final(return_df)
annualized_return = ep.annual_return(return_df)
max_drawdown = ep.max_drawdown(return_df)
sharpe_ratio = ep.sharpe_ratio(return_df)
# Print performance metrics
print(f"Total Return: {total_return:.2%}")
print(f"Annualized Return: {annualized_return:.2%}")
print(f"Max Drawdown: {max_drawdown:.2%}")
print(f"Sharpe Ratio: {sharpe_ratio:.4f}")